# 02 — Fetch GHCN (Illinois) from NOAA S3 → Parquet
This notebook reads **GHCN-Daily** metadata and inventory from NOAA's public S3, finds 4 Illinois stations with ≥30 years of record that **actually** have `csv/by_station` files, downloads and cleans their daily data, and writes a **local Parquet**: `data/ghcn_il_top4_daily.parquet`.

In [9]:
import pandas as pd, numpy as np
from pathlib import Path
import fsspec

S3_STATIONS_TXT   = "s3://noaa-ghcn-pds/ghcnd-stations.txt"
S3_INVENTORY_TXT  = "s3://noaa-ghcn-pds/ghcnd-inventory.txt"
S3_BY_STATION     = "s3://noaa-ghcn-pds/csv/by_station/{id}.csv"
STOR = {"anon": True}

OUTDIR = Path('../data'); OUTDIR.mkdir(parents=True, exist_ok=True)
OUT_PARQUET = OUTDIR / 'ghcn_il_top4_daily.parquet'
OUT_csv = OUTDIR / 'ghcn_il_top4_daily.csv'
print('Output:', OUT_PARQUET.resolve())

Output: /data/keeling/a/deffip2/ATMS_523/HW_3_Module_3/ATMS-523-Module-3-pandas-datetime-climate-deffip2/data/ghcn_il_top4_daily.parquet


## 1) Load stations (fixed-width) and inventory

help us decode the file \
to load the s3 data we should make sure we're working anonymously

In [2]:
colspecs = [(0,11),(12,20),(21,30),(31,37),(38,40),(41,71),(72,75),(76,79),(80,85)]
names = ['ID','LATITUDE','LONGITUDE','ELEVATION','STATE','NAME','GSN_FLAG','HCN_CRN_FLAG','WMO_ID']

stations = pd.read_fwf(S3_STATIONS_TXT, colspecs=colspecs, names=names, dtype={'ID':str,'STATE':str,'WMO_ID':str}, storage_options=STOR)
stations['NAME'] = stations['NAME'].str.strip(); stations['STATE'] = stations['STATE'].fillna('').str.strip()

inventory = pd.read_csv(
    S3_INVENTORY_TXT, sep=r'\s+', names=['ID','LAT','LON','ELEMENT','FIRSTYEAR','LASTYEAR'],
    dtype={'ID':str,'ELEMENT':str,'FIRSTYEAR':int,'LASTYEAR':int}, engine='python', storage_options=STOR
)

stations.head(), inventory.head()

(            ID  LATITUDE  LONGITUDE  ELEVATION STATE                   NAME  \
 0  ACW00011604   17.1167   -61.7833       10.1        ST JOHNS COOLIDGE FLD   
 1  ACW00011647   17.1333   -61.7833       19.2                     ST JOHNS   
 2  AE000041196   25.3330    55.5170       34.0          SHARJAH INTER. AIRP   
 3  AEM00041194   25.2550    55.3640       10.4                   DUBAI INTL   
 4  AEM00041217   24.4330    54.6510       26.8               ABU DHABI INTL   
 
   GSN_FLAG HCN_CRN_FLAG WMO_ID  
 0      NaN          NaN    NaN  
 1      NaN          NaN    NaN  
 2      GSN          NaN  41196  
 3      NaN          NaN  41194  
 4      NaN          NaN  41217  ,
             ID      LAT      LON ELEMENT  FIRSTYEAR  LASTYEAR
 0  ACW00011604  17.1167 -61.7833    TMAX       1949      1949
 1  ACW00011604  17.1167 -61.7833    TMIN       1949      1949
 2  ACW00011604  17.1167 -61.7833    PRCP       1949      1949
 3  ACW00011604  17.1167 -61.7833    SNOW       1949      194

## 2) Compute coverage; pick IL stations with ≥30 years

we use loc function in pandas to looking for certain information that meet the condition

In [ ]:
coverage = (inventory.groupby('ID', as_index=False)
                    .agg(first=('FIRSTYEAR','min'), last=('LASTYEAR','max'))
                    .assign(years=lambda d: d['last'] - d['first'] + 1))

il = (stations.loc[stations['STATE']=='IL', ['ID','NAME','STATE','LATITUDE','LONGITUDE','ELEVATION']]
              .merge(coverage, on='ID', how='inner'))
# how= inner --> tells you how we can do the merging 

il30 = il[il['years']>=30].copy()
il30.sort_values(['years','ID'], ascending=[False, True]).head(12)

,ID,NAME,STATE,LATITUDE,LONGITUDE,ELEVATION,first,last,years
2106,USC00117391,ROCK IS L&D 15,IL,41.5181,-90.5647,173.1,1866,2025,160
1692,USC00110137,ALTON - MELVIN PRICE L&D,IL,38.8669,-90.1489,123.4,1892,2025,134
2041,USC00116526,OTTAWA 4SW,IL,41.3281,-88.9097,166.1,1892,2025,134
1708,USC00110338,AURORA WATER,IL,41.7803,-88.3092,205.7,1893,2025,133
1747,USC00111329,CASEY,IL,39.2975,-87.9747,189.0,1893,2025,133
1799,USC00112193,DECATUR WTP,IL,39.8289,-88.9506,194.8,1893,2025,133
1806,USC00112348,DIXON WWTP,IL,41.8350,-89.5136,198.1,1893,2025,133
1812,USC00112483,DU QUOIN 4 SE,IL,37.9878,-89.1931,128.0,1893,2025,133
1821,USC00112679,EDWARDSVILLE 2 W,IL,38.8094,-90.0033,141.7,1893,2025,133
1855,USC00113335,GALVA,IL,41.1739,-90.0350,246.9,1893,2025,133


In [ ]:
coverage # coverage means we are calculating the record length so that's why we add 1 in the line of codes from previous cells

,ID,first,last,years
0,ACW00011604,1949,1949,1
1,ACW00011647,1957,1970,14
2,AE000041196,1944,2025,82
3,AEM00041194,1983,2025,43
4,AEM00041217,1983,2025,43
...,...,...,...,...
129617,ZI000067969,1962,1990,29
129618,ZI000067975,1951,2025,75
129619,ZI000067977,1968,1990,23
129620,ZI000067983,1951,2025,75


## 3) Probe `csv/by_station` and pick 4 that exist

In [5]:
fs = fsspec.filesystem('s3', **STOR)
candidates = il30.sort_values(['years','ID'], ascending=[False, True])['ID'].tolist()
picked, url_map = [], {}
for sid in candidates:
    url = S3_BY_STATION.format(id=sid)
    if fs.exists(url):
        picked.append(sid); url_map[sid] = url
    if len(picked)>=4: break
print('Picked:', picked)
url_map

Picked: ['USC00117391', 'USC00110137', 'USC00116526', 'USC00110338']


{'USC00117391': 's3://noaa-ghcn-pds/csv/by_station/USC00117391.csv',
 'USC00110137': 's3://noaa-ghcn-pds/csv/by_station/USC00110137.csv',
 'USC00116526': 's3://noaa-ghcn-pds/csv/by_station/USC00116526.csv',
 'USC00110338': 's3://noaa-ghcn-pds/csv/by_station/USC00110338.csv'}

## 4) Load, clean, pivot to wide, convert units

In [6]:
def load_station_daily(url: str) -> pd.DataFrame:
    df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])
    df['DATA_VALUE'] = df['DATA_VALUE'].replace(-9999, np.nan)
    wide = (df.pivot_table(index=['ID','DATE'], columns='ELEMENT', values='DATA_VALUE', aggfunc='first').reset_index())
    for c in ('TMAX','TMIN','TAVG'):
        if c in wide: wide[c] = wide[c]/10.0
    if 'PRCP' in wide: wide['PRCP'] = wide['PRCP']/10.0
    return wide.sort_values(['ID','DATE']).reset_index(drop=True)

frames = []
for sid in picked:
    w = load_station_daily(url_map[sid])
    frames.append(w); print(sid, w.shape)

daily = pd.concat(frames, ignore_index=True)
daily.head()

USC00117391 (19139, 14)


/tmp/ipykernel_127005/2081217567.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])


USC00110137 (29199, 22)


/tmp/ipykernel_127005/2081217567.py:2: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])


USC00116526 (48011, 25)


/tmp/ipykernel_127005/2081217567.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, storage_options=STOR, dtype={'ID':str,'ELEMENT':str}, parse_dates=['DATE'])


USC00110338 (47303, 26)


ELEMENT,ID,DATE,DAPR,MDPR,PRCP,SNOW,SNWD,TMAX,TMIN,TOBS,...,WT08,WT09,WT11,WT14,WT16,WT18,DASF,MDSF,WESD,EVAP
0,USC00117391,1866-02-01,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00117391,1866-02-02,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00117391,1866-02-03,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00117391,1866-02-04,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00117391,1866-02-05,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5) Write Parquet and quick verify

In [12]:
%time

first_cols = [c for c in ['ID','DATE','PRCP','TMAX','TMIN','TAVG','SNOW','SNWD'] if c in daily.columns]
daily = daily[first_cols + [c for c in daily.columns if c not in first_cols]]
daily.to_parquet(OUT_PARQUET, index=False)
print('Wrote:', OUT_PARQUET.resolve())
pd.read_parquet(OUT_PARQUET).groupby('ID').size()

CPU times: user 7 μs, sys: 1e+03 ns, total: 8 μs
Wall time: 13.1 μs
Wrote: /data/keeling/a/deffip2/ATMS_523/HW_3_Module_3/ATMS-523-Module-3-pandas-datetime-climate-deffip2/data/ghcn_il_top4_daily.parquet


ID
USC00110137    29199
USC00110338    47303
USC00116526    48011
USC00117391    19139
dtype: int64

In [13]:
%time

first_cols = [c for c in ['ID','DATE','PRCP','TMAX','TMIN','TAVG','SNOW','SNWD'] if c in daily.columns]
daily = daily[first_cols + [c for c in daily.columns if c not in first_cols]]
daily.to_csv(OUT_csv, index=False)
print('Wrote:', OUT_csv.resolve())
pd.read_csv(OUT_csv).groupby('ID').size()

CPU times: user 6 μs, sys: 1 μs, total: 7 μs
Wall time: 11.9 μs
Wrote: /data/keeling/a/deffip2/ATMS_523/HW_3_Module_3/ATMS-523-Module-3-pandas-datetime-climate-deffip2/data/ghcn_il_top4_daily.csv


ID
USC00110137    29199
USC00110338    47303
USC00116526    48011
USC00117391    19139
dtype: int64